In [1]:
from typing import Optional, Dict
from langchain_core.pydantic_v1 import BaseModel, Field

In [ ]:
from openai import OpenAI
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

In [2]:
theme = {
    "alias": {
        "radii": {
            "radius_100": "4px",
            "radius_200": "8px",
            "radius_300": "999px",
        },
        "background_100": "#fcf2eb",
        "background_200": "#fadfca",
        "background_300": "#ffdbbf",
        "background": {
            "contrast_100": "#f29750",
            "contrast_200": "#ed6b06",
        },
        "text_100": "#ed6b06",
        "text_200": "#261782",
        "text_300": "#ffffff",
        "border_100": "#ed6b06",
        "border": {
            "contrast_100": "#a2a2a2",
            "contrast_200": "#5e5851",
        },
        "states": {
            "success": "#ed6b06",  # have to be Hex because of hexToRGB()
            "neutral": "#fcf2eb",
            "warning": "#fc9a4e",
            "danger": "#db4223",
        },
    },
        "gradients": {
        # Merge existing gradients with new ones
        # **theme_gradients,
        "variant1": "linear-gradient(0deg, #120949, #120949)",
        "variant2": "linear-gradient(0deg, #FFFFFF 19.85%, #FFF 118.93%)",  # Secondary button
        "variant3": "linear-gradient(to right, #ed6b06, #ed6b06)",
    },
}

In [9]:


class Radii(BaseModel):
    radius_100: Optional[str] = Field(..., description="Radius size of 100")
    radius_200: Optional[str] = Field(..., description="Radius size of 200")
    radius_300: Optional[str] = Field(..., description="Radius size of 300")

class Background(BaseModel):
    contrast_100: Optional[str] = Field(..., description="Background contrast level 100")
    contrast_200: Optional[str] = Field(..., description="Background contrast level 200")

class Border(BaseModel):
    contrast_100: Optional[str] = Field(..., description="Border contrast level 100")
    contrast_200: Optional[str] = Field(..., description="Border contrast level 200")

class States(BaseModel):
    success: Optional[str] = Field(..., description="Color for success state")
    neutral: Optional[str] = Field(..., description="Color for neutral state")
    warning: Optional[str] = Field(..., description="Color for warning state")
    danger: Optional[str] = Field(..., description="Color for danger state")

class Gradient(BaseModel):
    variant1: Optional[str] = Field(..., description="Gradient variant 1")
    variant2: Optional[str] = Field(..., description="Gradient variant 2")
    variant3: Optional[str] = Field(..., description="Gradient variant 3")

class Alias(BaseModel):
    radii: Optional[Radii] = Field(..., description="Radii specifications")
    background_100: Optional[str] = Field(..., description="Primary background color")
    background_200: Optional[str] = Field(..., description="Secondary background color")
    background_300: Optional[str] = Field(..., description="Tertiary background color")
    background: Optional[Background] = Field(..., description="Background contrast specifications")
    text_100: Optional[str] = Field(..., description="Primary text color")
    text_200: Optional[str] = Field(..., description="Secondary text color")
    text_300: Optional[str] = Field(..., description="Tertiary text color")
    border_100: Optional[str] = Field(..., description="Primary border color")
    border: Optional[Border] = Field(..., description="Border contrast specifications")
    states: Optional[States] = Field(..., description="State colors")
    gradients: Optional[Gradient] = Field(..., description="Gradient specifications")

class Theme(BaseModel):
    alias: Optional[Alias] = Field(..., description="Alias specifications and semantic meanings")

# Example usage
example_theme = Theme(
    alias=Alias(
        radii=Radii(
            radius_100="4px",
            radius_200="8px",
            radius_300="999px",
        ),
        background_100="#fcf2eb",
        background_200="#fadfca",
        background_300="#ffdbbf",
        background=Background(
            contrast_100="#f29750",
            contrast_200="#ed6b06",
        ),
        text_100="#ed6b06",
        text_200="#261782",
        text_300="#ffffff",
        border_100="#ed6b06",
        border=Border(
            contrast_100="#a2a2a2",
            contrast_200="#5e5851",
        ),
        states=States(
            success="#ed6b06",
            neutral="#fcf2eb",
            warning="#fc9a4e",
            danger="#db4223",
        ),
        gradients=Gradient(
            variant1="linear-gradient(0deg, #120949, #120949)",
            variant2="linear-gradient(0deg, #FFFFFF 19.85%, #FFF 118.93%)",
            variant3="linear-gradient(to right, #ed6b06, #ed6b06)",
    ),
        )
)
example_theme

Theme(alias=Alias(radii=Radii(radius_100='4px', radius_200='8px', radius_300='999px'), background_100='#fcf2eb', background_200='#fadfca', background_300='#ffdbbf', background=Background(contrast_100='#f29750', contrast_200='#ed6b06'), text_100='#ed6b06', text_200='#261782', text_300='#ffffff', border_100='#ed6b06', border=Border(contrast_100='#a2a2a2', contrast_200='#5e5851'), states=States(success='#ed6b06', neutral='#fcf2eb', warning='#fc9a4e', danger='#db4223'), gradients=Gradient(variant1='linear-gradient(0deg, #120949, #120949)', variant2='linear-gradient(0deg, #FFFFFF 19.85%, #FFF 118.93%)', variant3='linear-gradient(to right, #ed6b06, #ed6b06)')))

In [4]:
from typing import Optional
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI


# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
output_parser = JsonOutputParser(pydantic_object=Theme)
format_instructions = output_parser.get_format_instructions()
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert extraction algorithm for design themes from pdf presentations.
            Only extract relevant information from the pdf output that relates to the creation of the appropriate alias object for styling a page.
            You will never return a null value for any section. If you do not know the value of an attribute asked to extract,
            return the best option for the context for that attribute's value."""
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ],
)

In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

In [11]:
llm = ChatOpenAI(model="gpt-4-0125-preview", temperature=0)

runnable = prompt | llm.with_structured_output(schema=Theme)

In [9]:
from langchain_community.document_loaders import PyPDFLoader


def extract_pdf_raw_contents(pdfpath):
    loader = PyPDFLoader(pdfpath)
    documents = loader.load()
    raw_contents = ""
    for doc in documents:
        raw_contents+=doc.page_content + "\n"
    
    return raw_contents
        
    
pdfpath = "./theme-base-client-file.pdf"

pdf_raw_contents = extract_pdf_raw_contents(pdfpath)
pdf_raw_contents[:50]

'1\nMontér Designmanual 2023\nDesignmanual\nSist oppda'

In [10]:
output = runnable.invoke({"text": pdf_raw_contents})
output

Theme(alias=Alias(radii=Radii(radius_100='10mm', radius_200='20mm', radius_300='30mm'), background_100='#F8E08E', background_200='#A59C94', background_300='#C3AD8B', background=Background(contrast_100='#101820', contrast_200='#EE3831'), text_100='#101820', text_200='#EE3831', text_300='#F3D03E', border_100='#101820', border=Border(contrast_100='#EE3831', contrast_200='#F3D03E'), states=States(success='#238823', neutral='#F8E08E', warning='#FFBF00', danger='#D2222D'), gradients=Gradient(variant1='linear-gradient(45deg, #F8E08E, #EE3831)', variant2='linear-gradient(45deg, #A59C94, #C3AD8B)', variant3='linear-gradient(45deg, #101820, #F3D03E)')))

In [11]:
output_json = output.json()
output_json

'{"alias": {"radii": {"radius_100": "10mm", "radius_200": "20mm", "radius_300": "30mm"}, "background_100": "#F8E08E", "background_200": "#A59C94", "background_300": "#C3AD8B", "background": {"contrast_100": "#101820", "contrast_200": "#EE3831"}, "text_100": "#101820", "text_200": "#EE3831", "text_300": "#F3D03E", "border_100": "#101820", "border": {"contrast_100": "#EE3831", "contrast_200": "#F3D03E"}, "states": {"success": "#238823", "neutral": "#F8E08E", "warning": "#FFBF00", "danger": "#D2222D"}, "gradients": {"variant1": "linear-gradient(45deg, #F8E08E, #EE3831)", "variant2": "linear-gradient(45deg, #A59C94, #C3AD8B)", "variant3": "linear-gradient(45deg, #101820, #F3D03E)"}}}'

In [12]:
import json

with open('alias-style.json', 'w+') as f:
    json.dump(output_json, f)